In [1]:
import matplotlib.pyplot as plt

import torchvision as tv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.optim as optim
import os
import time
import loadData
import itertools
from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda:0")

In [3]:
batchSize = 4
test_data = loadData.MyDataset(txt=f'test.txt', transform=None)
test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
print(test_loader)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(10064, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

net = Net()

In [5]:
path = './model_30dB_4个0一组.pkl'
net.load_state_dict(torch.load(path))
net = net.to(device)

In [6]:
transforms = tv.transforms.Compose([
    # transforms.CenterCrop(64),
    # transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [7]:
erroCounter = 0
realLabels = []
predictedLabels = []

In [9]:
correct = 0
total = 0
realLabel = []
predictedLabel = []

In [10]:
# x = torch.randn(4, 1, 640, 2).float()
# conv = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (10, 1))
# x = conv(x)
# x.shape
# # conv2 = nn.Conv2d(in_channels = 8, out_channels = 256, kernel_size = (3, 2))
# # x = conv2(x)
# # x.shape

In [11]:
 for data in test_loader:
    inputs, labels = data
    inputs, labels = inputs.cuda(), labels.cuda()
    inputs = torch.tensor(inputs, dtype=torch.float32, device = device)

    realLabel.append([int(label) for label in labels])
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, dim=1)
    predictedLabel.append([int(label) for label in predicted])
    total += labels.size(0)
    correct += (predicted == labels).sum()
    print("labels: ", labels)
    print("predicted: ", predicted)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


labels:  tensor([15], device='cuda:0')
predicted:  tensor([15], device='cuda:0')
